In [146]:
from collections import OrderedDict
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import re
import requests
import os
import string
from glob import glob
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 10000)

In [205]:
class html_table_scraper:
    '''class provides functions which can be used to scrape and merge html tables from baseball-reference'''
    
    def scrape_batting_tables(self,url):
        '''Scrapes "TEAM BATTING" html table into pandas data frame'''
        self.url = url

        #create bs object
        r = urllib.request.urlopen(url).read()
        soup = BeautifulSoup(r, "lxml")
        
        #get team name from url
        team_abbrev = re.split(r'[/.]', self.url)[6]

        #find table, find header, find rows
        table = soup.find('div', attrs={'class': 'overthrow table_container'})
        table_head = table.find('thead')
        table_body = table.find('tbody')

        #create table header list
        header = []    
        for th in table_head.findAll('th'):
            key = th.get_text()
            header.append(key)

        #find number of 'empty' rows
        endrows = 0
        for tr in table.findAll('tr'):
            if tr.findAll('th')[0].get_text() in (''):
                endrows += 1

        #find number of rows in table
        rows = len(table.findAll('tr'))
        rows -= endrows + 1  

        #create lists of row data, create ordered dictionary from header and row data...
        #...create list of dictionaries for data frame
        list_of_dicts = []
        for row in range(rows):
            the_row = []
            try:
                table_row = table.findAll('tr')[row]
                for tr in table_row:
                    value = tr.get_text()
                    the_row.append(value)
                od = OrderedDict(zip(header,the_row))
                list_of_dicts.append(od)
            except AttributeError:
                continue 

        #create df
        df = pd.DataFrame(list_of_dicts)

        #change column names to all uppercase for easy searching 
        df.columns = [col.upper() for col in df.columns]
        #add team column 
        df['TEAM_NAME'] = team_abbrev

        #strip all bad characters from NAME column to allow searching by name
        df['NAME'] = df['NAME'].str.replace(r'\(([^()]+)\)', '')
        df['NAME'] = df['NAME'].str.replace('*', '')
        df['NAME'] = df['NAME'].str.replace('#', '')

        #set index so it's easier to search
        df1 = df.set_index(keys=['NAME', 'TEAM_NAME'])
        return df1

    def get_team_names(self, base_url, year_as_str):
        '''returns a list of team name abbreviations used in baseball reference tables
           Args: years_as_str = str(/2016)'''
        self.base_url = base_url
        self.year_as_str = year_as_str

        base = 'http://www.baseball-reference.com/leagues/MLB/'
        ext = '.shtml'
        full_url = base + year_as_str + ext
        df = pd.read_html(full_url, flavor='html5lib', attrs={'class': 'sortable'})

        df_names = df[0]

        teams= df_names['Tm'].tolist()
        del teams[-2:]
        team_names = ['/' + i  for i in teams]

        return team_names
    
    def create_batting_links(self, base_url, team_names, years, extension):

        '''scrapes batting tables and saves to csv
        Args: str(base_url)
              list(team_name)
              list(year)
              str(extension)'''
        
        self.base_url = base_url
        self.team_names = team_names
        self.years = years 
        self.extension = extension

        links_lst = []
        for y in years:
            for n in team_names:
                links_lst.append(base_url + n + y + extension)

        length_list  =len(links_lst)
        count = 0

        while count < length_list:

            df = self.scrape_batting_tables(links_lst[count])
            file_name = links_lst[count][40:48].replace('/', '_')
            df.to_csv('batting_' + file_name + '.csv')
            count += 1
            print('Saved csv for team_year - ' + file_name)

            
    def get_player_names(self, fp):
        '''given a file path that has ALL batting stats for a specific team, 
           concatenates all csvs into a giant data frame -- folder must ONLY have batting csvs '''
    
        self.fp = fp
    
        #all csvs in same directory
        my_csv_paths = os.listdir(file_path)[1:-1]

        df_lst = []
        for csv in my_csv_paths:
            df = pd.read_csv(csv)
            df_lst.append(df)

        stats_all = pd.concat(df_lst)

        all_player_names = stats_all['NAME'].tolist()
        sorted_names = sorted(all_player_names, key=lambda x: x.split(" ")[-1])
        return sorted_names
    
    def get_names_from_master_list(self, master_list, letter):
        '''get a list of players by first letter of last name'''
        self.master_list = master_list
        self.letter = letter

        new_lst = []
        for i in master_list:
            split = i.split(" ")
            if split[1].startswith(letter):
                new_lst.append(i)
        return new_lst
    
    
    def scrape_salary_table(self, player_url):
        '''scrapes salary table as multi-index data frame, optional arg to save as csv'''
        self.player_url = player_url
        try: 
            page = requests.get(player_url).text
            table_code = page[page.find('<table class="sortable stats_table" id="br-salaries"'):]
            soup = BeautifulSoup(table_code, 'lxml')

            #second bs4 soup
            r = urllib.request.urlopen(player_url).read()
            normal_soup = BeautifulSoup(r)

            table_body  = soup.find('tbody')

            for i in normal_soup.find_all('h1'):
                for j in i:
                    player_name = i.get_text() 

            #find team name        
            for p in normal_soup.find_all('p'):
                for i in p.find_all('a'):
                    if '/teams' in i['href']: 
                        team_name = i['href'][7:10]

            # get position 
            for p in normal_soup.find_all('p'):
                if 'Position' in p.get_text():
                    pos = p.get_text()

            #height and weight        
            weight = normal_soup.find('span',{'itemprop':'weight'}).text
            height = normal_soup.find('span',{'itemprop':'height'}).text
            height2 = height[0] + "'" + height[2]

            split = pos.split(':')[1]
            position = split.strip()

            #this line is for salary
            sal = [j.get_text() for i in table_body.findAll('tr') for j in i.findAll('td') if j['data-stat'] == 'salary']
            salary_lst = [i.replace('$', '').replace('*', '') for i in sal]

            #this block is for years              
            years = table_body.findAll('th')                          
            years_lst = [i.get_text() for i in years]
            del years_lst[-1]

            #create a dictionary of dictionaries
            dd9 = {}
            dd9[player_name] = {}
            dd9[player_name]['years'] = years_lst
            dd9[player_name]['salary'] = salary_lst

            split = pd.DataFrame.from_dict(dd9, orient = 'index')

            #explodes a list into rows
            years_col = split.years.apply(lambda x: pd.Series(x)).unstack()
            salary_col= split.salary.apply(lambda x: pd.Series(x)).unstack()

            #concat one series to df then add other series to existing df
            df = years_col.to_frame()
            df['SALARY'] = salary_col
            df2 = df.reset_index()
            df3= df2.rename(columns = {0: 'YEARS', 'level_1': 'NAME'})

            del df3['level_0']
            df3['TEAM_NAME'] = team_name
            df3['POSITION'] = position
            df3['HEIGHT'] = height2
            df3['WEIGHT'] = weight

            df4 = df3.set_index(keys = ['NAME', 'TEAM_NAME', 'YEARS'])
            df4.to_csv(player_name  +'.csv')
            
        except Exception as e:
            print(str(e))
            pass
        
        
    def salary_crawl(self, player_name_lst):
        '''takes 1.5 hours for ~1400 players'''
        self.player_name_lst = player_name_lst

        url = 'http://www.baseball-reference.com/players/'
        alpha = string.ascii_lowercase ###change this

        abcc = [i + '/' for i in alpha]

        missing_names = []
        for letter in abcc:
            base_url = url + letter
            r = urllib.request.urlopen(base_url).read()
            soup = BeautifulSoup(r)
            for p in soup.find_all('p'):
                for i in p.find_all('a'):
                    if '/players' in i['href']:
                        if i.get_text() in player_name_lst:
                            shtml = i['href'][9:]
                            new_url = url + shtml
                            print(new_url)
                            self.scrape_salary_table(new_url)
                        else:
                            missing_names.append(i.get_text())
        return missing_names

        

        
table_scraper = html_table_scraper()

In [52]:
base_url = 'http://www.baseball-reference.com/leagues/MLB/'
year = '/2016'
team_names = table_scraper.get_team_names(base_url, year)
team_names

['/ARI',
 '/ATL',
 '/BAL',
 '/BOS',
 '/CHC',
 '/CHW',
 '/CIN',
 '/CLE',
 '/COL',
 '/DET',
 '/HOU',
 '/KCR',
 '/LAA',
 '/LAD',
 '/MIA',
 '/MIL',
 '/MIN',
 '/NYM',
 '/NYY',
 '/OAK',
 '/PHI',
 '/PIT',
 '/SDP',
 '/SEA',
 '/SFG',
 '/STL',
 '/TBR',
 '/TEX',
 '/TOR',
 '/WSN']

# Get batting tables for specific team-year combinations

In [90]:
#create specific team-year combinations
base_url = 'http://www.baseball-reference.com/teams'
years_lst2 = ['/2016']
ext = '.shtml'

team_links = table_scraper.create_batting_links(base_url, team_names, years_lst2, ext)
team_links

Saved csv for team_year - ARI_2016
Saved csv for team_year - ATL_2016
Saved csv for team_year - BAL_2016
Saved csv for team_year - BOS_2016
Saved csv for team_year - CHC_2016
Saved csv for team_year - CHW_2016
Saved csv for team_year - CIN_2016
Saved csv for team_year - CLE_2016
Saved csv for team_year - COL_2016
Saved csv for team_year - DET_2016
Saved csv for team_year - HOU_2016
Saved csv for team_year - KCR_2016
Saved csv for team_year - LAA_2016
Saved csv for team_year - LAD_2016
Saved csv for team_year - MIA_2016
Saved csv for team_year - MIL_2016
Saved csv for team_year - MIN_2016
Saved csv for team_year - NYM_2016
Saved csv for team_year - NYY_2016
Saved csv for team_year - OAK_2016
Saved csv for team_year - PHI_2016
Saved csv for team_year - PIT_2016
Saved csv for team_year - SDP_2016
Saved csv for team_year - SEA_2016
Saved csv for team_year - SFG_2016
Saved csv for team_year - STL_2016
Saved csv for team_year - TBR_2016
Saved csv for team_year - TEX_2016
Saved csv for team_y

# Must have already downloaded batting tables csvs for an entire season so get_player_names can read names into list

In [170]:
file_path = '/Users/Anthony/Desktop/python_projects/twitter/baseball/2016/test2'
player_names = get_player_names(file_path)
len(player_names)

1389

# scrapes player salary tables --  all players list (about 1400/1 hour)

In [ ]:
missing_players = table_scraper.salary_crawl(player_names)
missing_players

# merge player salary with team batting data

In [151]:
team_path = os.listdir('/Users/Anthony/Desktop/python_projects/twitter/baseball/2016/test2')
df_lst1 = []
for csv in team_path:
    df20 = pd.read_csv(csv)
    df_lst1.append(df20)

team_df = pd.concat(df_lst1)
team_df.head()

,NAME,TEAM_NAME,RK,POS,AGE,G,PA,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP,SH,SF,IBB
0,Welington Castillo,ARI,1,C,29,113,457,416,41,110,24,0,14,68,2,0,33,121,0.264,0.322,0.423,0.745,91.0,176,5,4,0,4,3
1,Paul Goldschmidt,ARI,2,1B,28,158,705,579,106,172,33,3,24,95,32,5,110,150,0.297,0.411,0.489,0.899,133.0,283,14,7,0,8,15
2,Jean Segura,ARI,3,2B,26,153,694,637,102,203,41,7,20,64,33,10,39,101,0.319,0.368,0.499,0.867,122.0,318,6,12,4,2,1
3,Nick Ahmed,ARI,4,SS,26,90,308,284,26,62,9,1,4,20,5,2,15,58,0.218,0.265,0.299,0.564,46.0,85,9,4,2,3,3
4,Jake Lamb,ARI,5,3B,25,151,594,523,81,130,31,9,29,91,6,1,64,154,0.249,0.332,0.509,0.840,114.0,266,13,3,0,4,5


In [154]:
sal_path = os.listdir('/Users/Anthony/Desktop/python_projects/twitter/baseball/2016/test')
df_lst = []
for csv in sal_path:
    df1 = pd.read_csv(csv)
    df_lst.append(df1)

sal_df = pd.concat(df_lst)
del sal_df['Unnamed: 0']

In [155]:
sal_df1 = sal_df[sal_df['years'] == 2016.0] #get only 2016 
sal_df2 = sal_df1.dropna(axis=0)   #drop any row with empty cell
sal_df3 = sal_df2[sal_df2['salary'] > '0']
sal_df4 = sal_df3.rename(columns = {'name': 'NAME', 'team_name': 'TEAM_NAME'}) # rename columns to merge 
sal_df4.head()

,height,NAME,position,salary,TEAM_NAME,weight,years
5,6'2,A.J. Ellis,Catcher,"4,500,000",MIA,225lb,2016.0
3,5'1,A.J. Ramos,Pitcher,"3,400,000",MIA,200lb,2016.0
0,6'5,Aaron Altherr,Outfielder,"515,500",PHI,215lb,2016.0
3,6'1,Aaron Hicks,Outfielder,"574,000",NYY,202lb,2016.0
10,5'1,Aaron Hill,"Second Baseman, Third Baseman and Shortstop","12,000,000",SFG,200lb,2016.0


# merge salary_df4 only if name and team column are in team_df 

In [162]:
combined_df = pd.merge(sal_df4, team_df, on=['NAME', 'TEAM_NAME'], how='outer')
len(combined_df)

1768

In [163]:
c2 = combined_df.fillna(0)
has_salary = c2[c2['salary'] != 0]
len(has_salary)

749

In [164]:
has_ab = has_salary[has_salary['AB'] > 10]
has_ab.head()

,height,NAME,position,salary,TEAM_NAME,weight,years,RK,POS,AGE,G,PA,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP,SH,SF,IBB
2,6'5,Aaron Altherr,Outfielder,"515,500",PHI,215lb,2016.0,11.0,RF,25.0,57.0,227.0,198.0,23.0,39.0,6.0,0.0,4.0,22.0,7.0,2.0,23.0,69.0,0.197,0.300,0.288,0.587,60.0,57.0,4.0,6.0,0.0,0.0,2.0
3,6'1,Aaron Hicks,Outfielder,"574,000",NYY,202lb,2016.0,8.0,RF,26.0,123.0,361.0,327.0,32.0,71.0,13.0,1.0,8.0,31.0,3.0,4.0,30.0,68.0,0.217,0.281,0.336,0.617,65.0,110.0,7.0,0.0,1.0,3.0,1.0
6,6'2,Aaron Nola,Pitcher,"517,500",PHI,195lb,2016.0,27.0,P,23.0,20.0,35.0,30.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,17.0,0.033,0.147,0.067,0.214,-39.0,2.0,1.0,1.0,1.0,0.0,0.0
8,6'3,Adam Conley,Pitcher,"512,500",MIA,200lb,2016.0,26.0,P,26.0,25.0,47.0,41.0,4.0,5.0,0.0,0.0,0.0,3.0,0.0,0.0,1.0,22.0,0.122,0.140,0.122,0.261,-27.0,5.0,1.0,0.0,4.0,1.0,0.0
9,6'1,Adam Duvall,Leftfielder and First Baseman,"510,000",CIN,215lb,2016.0,6.0,LF,27.0,150.0,608.0,552.0,85.0,133.0,31.0,6.0,33.0,103.0,6.0,5.0,41.0,164.0,0.241,0.297,0.498,0.795,106.0,275.0,7.0,6.0,0.0,8.0,1.0


In [165]:
len(has_ab)

279

In [125]:
has_ab.to_csv('name_of_file.csv')

,height,NAME,position,salary,TEAM_NAME,weight,years,RK,POS,AGE,G,PA,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP,SH,SF,IBB
2,6'5,Aaron Altherr,Outfielder,"515,500",PHI,215lb,2016.0,11.0,RF,25.0,57.0,227.0,198.0,23.0,39.0,6.0,0.0,4.0,22.0,7.0,2.0,23.0,69.0,0.197,0.300,0.288,0.587,60.0,57.0,4.0,6.0,0.0,0.0,2.0
3,6'1,Aaron Hicks,Outfielder,"574,000",NYY,202lb,2016.0,8.0,RF,26.0,123.0,361.0,327.0,32.0,71.0,13.0,1.0,8.0,31.0,3.0,4.0,30.0,68.0,0.217,0.281,0.336,0.617,65.0,110.0,7.0,0.0,1.0,3.0,1.0
6,6'2,Aaron Nola,Pitcher,"517,500",PHI,195lb,2016.0,27.0,P,23.0,20.0,35.0,30.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,17.0,0.033,0.147,0.067,0.214,-39.0,2.0,1.0,1.0,1.0,0.0,0.0
7,6'4,Aaron Sanchez,Pitcher,"517,800",TOR,215lb,2016.0,23.0,P,23.0,30.0,10.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,0.000,0.100,0.000,0.100,-69.0,0.0,0.0,0.0,0.0,0.0,0.0
8,6'3,Adam Conley,Pitcher,"512,500",MIA,200lb,2016.0,26.0,P,26.0,25.0,47.0,41.0,4.0,5.0,0.0,0.0,0.0,3.0,0.0,0.0,1.0,22.0,0.122,0.140,0.122,0.261,-27.0,5.0,1.0,0.0,4.0,1.0,0.0
9,6'1,Adam Duvall,Leftfielder and First Baseman,"510,000",CIN,215lb,2016.0,6.0,LF,27.0,150.0,608.0,552.0,85.0,133.0,31.0,6.0,33.0,103.0,6.0,5.0,41.0,164.0,0.241,0.297,0.498,0.795,106.0,275.0,7.0,6.0,0.0,8.0,1.0
11,6'2,Adam Jones,Centerfielder,"16,333,333",BAL,215lb,2016.0,7.0,CF,30.0,152.0,672.0,619.0,86.0,164.0,19.0,0.0,29.0,83.0,2.0,0.0,39.0,115.0,0.265,0.310,0.436,0.746,96.0,270.0,13.0,5.0,1.0,8.0,2.0
15,6'7,Adam Wainwright,Pitcher,"19,500,000",STL,235lb,2016.0,22.0,P,34.0,34.0,69.0,62.0,6.0,13.0,7.0,1.0,2.0,18.0,0.0,0.0,2.0,17.0,0.210,0.234,0.452,0.686,79.0,28.0,1.0,0.0,5.0,0.0,0.0
18,6'0,Addison Russell,Shortstop and Second Baseman,"527,000",CHC,200lb,2016.0,4.0,SS,22.0,151.0,598.0,525.0,67.0,125.0,25.0,3.0,21.0,95.0,5.0,1.0,55.0,135.0,0.238,0.321,0.417,0.738,98.0,219.0,11.0,12.0,0.0,6.0,6.0
19,6'0,Adeiny Hechavarria,Shortstop,"2,625,000",MIA,195lb,2016.0,4.0,SS,27.0,155.0,547.0,508.0,52.0,120.0,17.0,6.0,3.0,38.0,1.0,0.0,33.0,73.0,0.236,0.283,0.311,0.594,63.0,158.0,10.0,1.0,2.0,3.0,7.0
